# Dependencies

## Installation

In [ ]:
%pip install pip --upgrade -q
%pip install pandas -q
%pip install numpy -q
%pip install matplotlib -q
%pip install folium -q
%pip install geopy -q
%pip install geopandas -q

## Importing

In [ ]:
import pandas as pd
import folium
import geopandas as gpd

# Obtaining Brazilian Municipalities Geo Data

In [ ]:
EPSG_CODE_FOR_FOLIUM = 4326
ORIGINAL_GEO_DATA_FILE_PATH = "data/geo/BR_Municipios_2023/BR_Municipios_2023.shp"
municipality_geo_file = gpd.read_file(ORIGINAL_GEO_DATA_FILE_PATH)
municipality_geo_file = municipality_geo_file

In [ ]:
# Getting municipalities centroid's lats and longitudes
geo_data_latitudes = municipality_geo_file.to_crs(epsg=EPSG_CODE_FOR_FOLIUM).geometry.centroid.x.astype(str)
geo_data_longitudes = municipality_geo_file.to_crs(epsg=EPSG_CODE_FOR_FOLIUM).geometry.centroid.y.astype(str)

# Excluding the last digit of the IBGE code, which is the verification digit
geo_ibge_code = municipality_geo_file["CD_MUN"].apply(lambda x: x.strip()[:6])

# Getting the geometry of the municipalities
geo_geometry = municipality_geo_file["geometry"].simplify(tolerance=0.001).to_crs(epsg=EPSG_CODE_FOR_FOLIUM)


In [ ]:
# Creating a GeoDataFrame with the relevant data for the municipalities plot
geo_data = pd.DataFrame({
    "IBGE-code": geo_ibge_code,
    "IBGE-state-code": geo_ibge_code.apply(lambda x: x[:2]),
    "IBGE-municipality-code": geo_ibge_code.apply(lambda x: x[2:]),
    "municipality-state": municipality_geo_file["NM_UF"].astype(str),
    "municipality-name": municipality_geo_file["NM_MUN"].astype(str),
    "latitude": geo_data_latitudes,
    "longitude": geo_data_longitudes
})            

# Preview
geo_data.head(5)

In [ ]:
# Saving the GeoDataFrame to a GeoJSON file
GENERATED_GEO_DATA_FILE_PATH = "./data/generated/geo/municipalities_geodata.geojson"
geo_data = gpd.GeoDataFrame(geo_data, geometry=geo_geometry, crs=EPSG_CODE_FOR_FOLIUM)
geo_data.to_file(GENERATED_GEO_DATA_FILE_PATH, driver='GeoJSON', encoding='utf-8')

In [ ]:
# File preview
geo_data = gpd.read_file(GENERATED_GEO_DATA_FILE_PATH)
geo_data.head(5)

In [ ]:
#Preview Folium map
BRAZILIA_CORDINATES = [-15.7801, -47.9292]
m = folium.Map(location=BRAZILIA_CORDINATES, zoom_start=4, tiles='cartodb positron')
folium.GeoJson(
    geo_data,
    name="All Municipalities",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
        "fillColor": "white",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(m)
folium.LayerControl().add_to(m)

m